In [1]:
!pip install sklearn
import sklearn


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [8 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\brice\AppData\Local\Temp\pip-install-cwtnqfls\sklearn_acbe9ecaaea74e6a8691f9cbc274184d\setup.py", line 10, in <module>
      LONG_DESCRIPTION = f.read()
    File "C:\Users\brice\anaconda3\lib\encodings\cp1252.py", line 23, in decode
      return codecs.charmap_decode(input,self.errors,decoding_table)[0]
  UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above

  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import html
import re
import chardet

path = "data"
data = []

def clean_text(text):
    # Supprime les numéros de pages (ex: "p. 574")
    text = re.sub(r'\bp\.\s+\d+\b', '', text)
    return text.strip()

# Parcourir tous les fichiers du dossier "data"
for file in os.listdir(path):
    path_file = os.path.join(path, file)

    # Vérifie si le chemin correspond à un fichier
    if os.path.isfile(path_file):
        # Détecte l'encodage du fichier
        with open(path_file, 'rb') as fp:
            encoding_result = chardet.detect(fp.read())

        # Utilise l'encodage détecté pour ouvrir le fichier
        with open(path_file, encoding=encoding_result['encoding']) as fp:
            soup = BeautifulSoup(fp, "html.parser")

        # Sélectionne les paragraphes sans classes spécifiques, ou avec des classes/id "STDIT" ou "id"
        paragraphes = soup.select("p:not([class]):not([style]), p[id], p[class=STDIT]")

        # Parcourt les paragraphes pour effectuer le nettoyage
        for p in paragraphes:
            # Remplace les sauts de ligne (<br>) par des espaces
            for br in p.find_all("br"):
                br.replace_with(" ")

            # Supprime les balises <span>
            for span in p.find_all("span"):
                span.replace_with(" ")

            # Supprime les balises <sup>
            for sup in p.find_all("sup"):
                sup.replace_with(" ")

            # Supprime les balises <i> en conservant le contenu
            for i in p.find_all("i"):
                i.unwrap()

            # Récupère le texte du paragraphe et effectue des nettoyages supplémentaires
            texte = p.get_text()
            texte = html.unescape(texte)
            texte = clean_text(texte)

            # Extrait le label, le nom du livre et le nom du fichier à partir du nom du fichier
            label = file.split("-")[0]
            livre = file.split("-", 1)[1].rsplit(".", 1)[0]

            # Ajoute les données extraites à la liste "data"
            data.append({"label": label, "texte": texte, "livre": livre, "fichier": file})

# Convertit la liste "data" en DataFrame pandas
df = pd.DataFrame(data)

In [3]:
# Récupère les valeurs uniques de la colonne 'fichier' dans le DataFrame df
df['fichier'].unique()

array(['balzac-comedie_humaine-vol01.htm',
       'balzac-comedie_humaine-vol02.htm',
       'balzac-comedie_humaine-vol03.htm',
       'flaubert-bouvard_pecuchet-pg14157.html',
       'flaubert-education_sentimentale_01.htm',
       'flaubert-madame_bovary-pg14155.html', 'flaubert-salammbo.htm',
       'maupassant-fort.htm', 'maupassant-pierre_et_jean.htm',
       'maupassant-une_vie-pg17457.html', 'sand-chateau_desertes.htm',
       'sand-fadette.htm', 'sand-lucrezia_floriani.htm',
       'sand-maitres_sonneurs-pg23582.html',
       'sand-mare_diable-pg23582.html', 'sand-meunier_angibault.htm',
       'zola-argent-pg6497.html', 'zola-assommoir-pg6497.html',
       'zola-bonheur_dames-pg16852.html', 'zola-germinal-pg5711.html',
       'zola-paris.htm', 'zola-ventre_paris-pg6470.html'], dtype=object)

In [4]:
# Récupère les valeurs uniques de la colonne 'livre' dans le DataFrame df
df['livre'].unique()

array(['comedie_humaine-vol01', 'comedie_humaine-vol02',
       'comedie_humaine-vol03', 'bouvard_pecuchet-pg14157',
       'education_sentimentale_01', 'madame_bovary-pg14155', 'salammbo',
       'fort', 'pierre_et_jean', 'une_vie-pg17457', 'chateau_desertes',
       'fadette', 'lucrezia_floriani', 'maitres_sonneurs-pg23582',
       'mare_diable-pg23582', 'meunier_angibault', 'argent-pg6497',
       'assommoir-pg6497', 'bonheur_dames-pg16852', 'germinal-pg5711',
       'paris', 'ventre_paris-pg6470'], dtype=object)

In [5]:
# Liste des noms des livres
livres = [    'comedie_humaine-vol01', 'comedie_humaine-vol02',    'comedie_humaine-vol03', 'bouvard_pecuchet-pg14157',    'education_sentimentale_01', 'madame_bovary-pg14155', 'salammbo',    'fort', 'pierre_et_jean', 'une_vie-pg17457', 'chateau_desertes',    'fadette', 'lucrezia_floriani', 'maitres_sonneurs-pg23582',    'mare_diable-pg23582', 'meunier_angibault', 'argent-pg6497',    'assommoir-pg6497', 'bonheur_dames-pg16852', 'germinal-pg5711',    'paris', 'ventre_paris-pg6470']

# Crée un dictionnaire avec les noms des livres comme clés et les DataFrames correspondants comme valeurs
dfs_par_livre = {livre: df[df["livre"] == livre] for livre in livres}

# Assignation des DataFrames pour chaque livre
df_livre_1 = dfs_par_livre["comedie_humaine-vol01"]
df_livre_2 = dfs_par_livre["comedie_humaine-vol02"]
df_livre_3 = dfs_par_livre["comedie_humaine-vol03"]
df_livre_4 = dfs_par_livre["bouvard_pecuchet-pg14157"]
df_livre_5 = dfs_par_livre["education_sentimentale_01"]
df_livre_6 = dfs_par_livre["madame_bovary-pg14155"]
df_livre_7 = dfs_par_livre["salammbo"]
df_livre_8 = dfs_par_livre["fort"]
df_livre_9 = dfs_par_livre["pierre_et_jean"]
df_livre_10 = dfs_par_livre["une_vie-pg17457"]
df_livre_11 = dfs_par_livre["chateau_desertes"]
df_livre_12 = dfs_par_livre["fadette"]
df_livre_13 = dfs_par_livre["lucrezia_floriani"]
df_livre_14 = dfs_par_livre["maitres_sonneurs-pg23582"]
df_livre_15 = dfs_par_livre["mare_diable-pg23582"]
df_livre_16 = dfs_par_livre["meunier_angibault"]
df_livre_17 = dfs_par_livre["argent-pg6497"]
df_livre_18 = dfs_par_livre["assommoir-pg6497"]
df_livre_19 = dfs_par_livre["bonheur_dames-pg16852"]
df_livre_20 = dfs_par_livre["germinal-pg5711"]
df_livre_21 = dfs_par_livre["paris"]
df_livre_22 = dfs_par_livre["ventre_paris-pg6470"]

In [6]:
# Parcourt chaque ligne du DataFrame df_livre_22
for i in range(len(df_livre_22)):
    # Affiche l'index et le texte correspondant pour chaque ligne
    print(f"{i}: {df_livre_22['texte'].iloc[i]}")

0: The Project Gutenberg EBook of Le Ventre de Paris, by Émile Zola
1: This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license
2: Title: Le Ventre de Paris
3: Author: Émile Zola
4: Posting Date: July 20, 2014 [EBook #6470]
Release Date: September, 2004
[This file was first posted on December 18, 2002]
[Last updated: September 16, 2014]
5: Language: French
6: *** START OF THIS PROJECT GUTENBERG EBOOK LE VENTRE DE PARIS ***
7: Produced by Philippe Chavin, Carlo Traverso, Juliet
Sutherland, Charles Franks and the Online Distributed
Proofreading Team. Image files courtesy of gallica.bnf.fr.
8: Au milieu du grand silence, et dans le désert de l'avenue, les
voitures de maraîchers montaient vers Paris, avec les cahots rhythmés
de leurs roues, dont les échos battaient les façades des maisons,
end

avait une autre femme…
1448: — Ça ne m'étonne pas, interrompit la Sarriette; ces hommes maigres
sont de fiers hommes.
1449: — Oui, et pas jeune encore, cette femme. Vous savez, quand un homme
en veut, il en veut; il en ramasserait par terre… Madame Verlaque,
la femme de l'ancien inspecteur, vous la connaissez bien, cette dame
toute jaune…
1450: Mais les deux autres se récrièrent. Ce n'était pas possible. Madame 

Verlaque était abominable. Alors mademoiselle Saget s'emporta.
1451: — Quand je vous le dis! Accusez-moi de mentir, n'est-ce pas?… On a
des preuves, on a trouvé des lettres de cette femme, tout un paquet de
lettres, dans lesquelles elle lui demandait de l'argent, des dix et
vingt francs à la fois. C'est clair, enfin… À eux deux, ils auront
fait mourir le mari.
1452: La Sarriette et madame Lecoeur furent convaincues. Mais elles
perdaient patience. Il y avait plus d'une heure qu'elles attendaient
sur le trottoir. Elles disaient que, pendant ce temps, on les volait
peut-être, à l

In [7]:
# Définit une fonction pour nettoyer le DataFrame
def clean_dataframe(df, start, end):
    # Extrait les lignes du DataFrame entre les index 'start' et 'end'
    df = df.iloc[start:end]
    # Supprime les lignes dont le texte est composé uniquement d'espaces
    df = df[df["texte"].str.strip() != ""]
    # Retourne le DataFrame nettoyé
    return df

# Nettoie chaque DataFrame en utilisant la fonction 'clean_dataframe' avec les index de début et de fin appropriés
dfs_par_livre["comedie_humaine-vol01"] = clean_dataframe(df_livre_1, 1, 2707)
dfs_par_livre["comedie_humaine-vol02"] = clean_dataframe(df_livre_2, 0, 1284)
dfs_par_livre["comedie_humaine-vol03"] = clean_dataframe(df_livre_3, 0, 2410)
dfs_par_livre["bouvard_pecuchet-pg14157"] = clean_dataframe(df_livre_4, 11, 3283)
dfs_par_livre["education_sentimentale_01"] = clean_dataframe(df_livre_5, 0, 2313)
dfs_par_livre["madame_bovary-pg14155"] = clean_dataframe(df_livre_6, 11, 3009)
dfs_par_livre["salammbo"] = clean_dataframe(df_livre_7, 2, 2154)
dfs_par_livre["fort"] = clean_dataframe(df_livre_8, 0, len(df_livre_8))
dfs_par_livre["pierre_et_jean"] = clean_dataframe(df_livre_9, 0, len(df_livre_9))
dfs_par_livre["une_vie-pg17457"] = clean_dataframe(df_livre_10, 10, 2544)
dfs_par_livre["chateau_desertes"] = clean_dataframe(df_livre_11, 0, 606)
dfs_par_livre["fadette"] = clean_dataframe(df_livre_12, 1, 721)
dfs_par_livre["lucrezia_floriani"] = clean_dataframe(df_livre_13, 0, len(df_livre_13))
dfs_par_livre["maitres_sonneurs-pg23582"] = clean_dataframe(df_livre_14, 0, len(df_livre_14))
dfs_par_livre["mare_diable-pg23582"] = clean_dataframe(df_livre_15, 19, 732)
dfs_par_livre["meunier_angibault"] = clean_dataframe(df_livre_16, 0, len(df_livre_16))
dfs_par_livre["argent-pg6497"] = clean_dataframe(df_livre_17, 0, len(df_livre_17))
dfs_par_livre["assommoir-pg6497"] = clean_dataframe(df_livre_18, 14, 2198)
dfs_par_livre["bonheur_dames-pg16852"] = clean_dataframe(df_livre_19, 11, 2993)
dfs_par_livre["germinal-pg5711"] = clean_dataframe(df_livre_20, 24, 3216)
dfs_par_livre["paris"] = clean_dataframe(df_livre_21, 0, len(df_livre_21))
dfs_par_livre["ventre_paris-pg6470"] = clean_dataframe(df_livre_22, 8, 1559)


In [8]:
clean_df = pd.concat(dfs_par_livre.values())

In [9]:
clean_df

,label,texte,livre,fichier
1,balzac,"Balzac naquit à Tours le 16 mai 1799, le jour ...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm
2,balzac,La première partie de Louis Lambert contient s...,comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm
3,balzac,Balzac souffrit prodigieusement dans ce collég...,comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm
4,balzac,"Le résultat de ces travaux cachés, de ces médi...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm
5,balzac,"A propos de vers, consignons ici un petit rens...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm
...,...,...,...,...
46437,zola,"Alors, Claude leur montra le poing. Il était e...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html
46438,zola,"À sa droits, la belle Normande, la belle madam...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html
46439,zola,"À sa gauche, la belle Lisa, au seuil de la cha...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html
46440,zola,"Puis, toutes deux se penchèrent. La belle mada...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html


In [11]:
import spacy
import re
import nltk
from nltk.util import ngrams

nltk.download('punkt')
nltk.download('stopwords')


# Charger le modèle de langue française de spaCy
nlp = spacy.load("fr_core_news_sm")

def preprocess_text(text, n=1, remove_stop_words=True, lemmatize=True, lowercase=True):
    # Suppression de la ponctuation spécifique
    punctuation = "«»—()[]"
    for char in punctuation:
        text = text.replace(char, "")

    # Tokenisation
    tokens = [token.text for token in nlp(text)]

    # Suppression des ponctuations après la tokenisation
    tokens = [token for token in tokens if token.isalnum()]

    # Mise en minuscule si lowercase est True
    if lowercase:
        tokens = [token.lower() for token in tokens]

    # Lemmatisation avec spaCy si lemmatize est True
    if lemmatize:
        spacy_text = nlp(" ".join(tokens))
        tokens = [token.lemma_ for token in spacy_text]

    # Suppression des mots vides
    if remove_stop_words:
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('french'))
        tokens = [token for token in tokens if token.lower() not in stop_words]

    # N-grammes
    n_grams = list(ngrams(tokens, n))
    n_grams = [' '.join(grams) for grams in n_grams]

    # Jointure des mots pour créer le texte pré-traité
    preprocessed_text = ' '.join(n_grams)
    
    return preprocessed_text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brice\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
preprocessed_df = clean_df
preprocessed_df['texte_preprocessed'] = clean_df['texte'].apply(preprocess_text)

In [13]:
preprocessed_df

,label,texte,livre,fichier,texte_preprocessed
1,balzac,"Balzac naquit à Tours le 16 mai 1799, le jour ...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm,balzac naître tour 16 mai 1799 jour fête saint...
2,balzac,La première partie de Louis Lambert contient s...,comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm,premier partie louis lambert contenir temps vi...
3,balzac,Balzac souffrit prodigieusement dans ce collég...,comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm,balzac souffrir prodigieusement collége vendôm...
4,balzac,"Le résultat de ces travaux cachés, de ces médi...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm,résultat travail cacher méditation prendre tem...
5,balzac,"A propos de vers, consignons ici un petit rens...",comedie_humaine-vol01,balzac-comedie_humaine-vol01.htm,avoir propos vers consignon ici petit renseign...
...,...,...,...,...,...
46437,zola,"Alors, Claude leur montra le poing. Il était e...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html,alors claude montrer poing être exaspérer fête...
46438,zola,"À sa droits, la belle Normande, la belle madam...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html,droit bel normande bel monsieur lebigre comme ...
46439,zola,"À sa gauche, la belle Lisa, au seuil de la cha...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html,gauche bel lisa seuil charcuterie tenir tout l...
46440,zola,"Puis, toutes deux se penchèrent. La belle mada...",ventre_paris-pg6470,zola-ventre_paris-pg6470.html,pouvoir tout deux pencher bel monsieur lebigre...


In [17]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

# Grouper les textes pré-traités par auteur (label)
grouped_text = preprocessed_df.groupby('label')['texte_preprocessed'].apply(' '.join).reset_index()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(grouped_text['texte_preprocessed'])
feature_names = vectorizer.get_feature_names()

# Créez un dictionnaire pour chaque auteur avec les mots et leurs scores TF-IDF
tfidf_dicts = []
for i, label in enumerate(grouped_text['label']):
    feature_index = tfidf_matrix[i, :].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    word_score_dict = {feature_names[i]: score for i, score in tfidf_scores}
    tfidf_dicts.append((label, word_score_dict))

# Afficher un nuage de mots pour chaque auteur (label)
for label, word_score_dict in tfidf_dicts:
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100)
    wordcloud.generate_from_frequencies(word_score_dict)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Nuage de mots pour {label}')
    plt.show()

ModuleNotFoundError: No module named 'wordcloud'